Proyect

#Solución Avanzada con LightGBM: Ingeniería de Características e Interacciones No Lineales

1. Resumen de la Estrategia
Esta solución implementa un modelo de clasificación basado en Gradient Boosting (LightGBM), diseñado específicamente para tabular datos socioeconómicos con alta eficiencia. A diferencia de enfoques tradicionales, esta estrategia prioriza la Ingeniería de Características (Feature Engineering) sobre la complejidad bruta del modelo, buscando capturar patrones de comportamiento mediante la interacción de variables.

2. Innovaciones Clave (El "Por qué" funciona)
Manejo Nativo de Categorías: Se utiliza la capacidad de LightGBM para procesar variables categóricas (como Departamento o Colegio) mediante algoritmos de partición óptima (Fisher), superando al tradicional One-Hot Encoding o Label Encoding que pierden información semántica.

Captura de Interacciones Socioeconómicas: Se han creado variables sintéticas que cruzan dimensiones clave:

TECH_SCORE (Índice de Riqueza Tecnológica): Agregación de activos (Internet + PC + Lavadora) para cuantificar el nivel de recursos del estudiante.

EDU_x_ESTRATO (Interacción No Lineal): Multiplicación del nivel educativo de los padres por el estrato. Esto permite al modelo entender que el impacto de la educación parental se potencia en estratos altos, capturando una realidad social compleja de Colombia.

Preservación de Jerarquía Ordinal: Variables como Nivel Educativo y Estrato fueron mapeadas numéricamente respetando su orden lógico (ej: Postgrado > Primaria), facilitando al árbol de decisión encontrar cortes de separación óptimos.

3. Configuración del Entrenamiento
Algoritmo: LightGBM (Histogram-based Gradient Boosting).

Validación: StratifiedKFold (5 particiones) para garantizar que la distribución de las clases (Bajo, Medio, Alto) se mantenga constante en entrenamiento y validación.

Optimización: Se utiliza un Learning Rate de 0.05 con Early Stopping, logrando un equilibrio entre convergencia rápida y capacidad de generalización para evitar el sobreajuste (overfitting).

In [1]:
!pip install rlxutils


  Preparing metadata (setup.py) ... done
  Created wheel for rlxutils: filename=rlxutils-0.1.10-py3-none-any.whl size=11100 sha256=eb3352a59a3db1229b85f911a9117ec164fdaf4ae2214d2567d1ae9153ebf4d2
  Stored in directory: /root/.cache/pip/wheels/fa/8f/e3/7c998c7a46a028383fd3c53494f18d81d52b7368c6b33e4ab6
Successfully built rlxutils


In [2]:
!pip install kaggle
import os

os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip -d data



  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 388MB/s]
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: data/submission_example.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


In [3]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia


udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!pip install -q kaggle
import os

# Configura la ruta del token
os.environ['KAGGLE_CONFIG_DIR'] = '.'

# Permisos de seguridad para el archivo
!chmod 600 ./kaggle.json

# Descarga el dataset del concurso
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia


udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


#Modelo LightGBM

In [5]:
!pip install lightgbm pandas scikit-learn scipy

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import mode
import warnings

# Configuración visual
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

print("Librerías cargadas correctamente.")

Librerías cargadas correctamente.


#Mapeos y función de preprocesamiento

In [6]:
# 2. DEFINICIÓN Y PREPROCESAMIENTO "TURBO"

# Mapas
edu_map = {
    'No sabe': 0, 'No Aplica': 0, 'Ninguno': 0,
    'Primaria incompleta': 1, 'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
    'Postgrado': 9
}
estrato_map = {'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6, 'Sin Estrato': 0}
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
inv_target_map = {v: k for k, v in target_map.items()}

def preprocess_turbo(df):
    df = df.copy()

    # 1. Numéricos básicos
    cols_edu = ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']
    for col in cols_edu:
        if col in df.columns:
            df[col] = df[col].map(edu_map).fillna(1)

    if 'F_ESTRATOVIVIENDA' in df.columns:
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(estrato_map).fillna(2)

    # 2. Binarias
    binarias = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENECOMPUTADOR', 'E_PAGOMATRICULAPROPIO']
    for col in binarias:
        if col in df.columns:
            df[col] = df[col].map({'Si': 1, 'No': 0, 'S': 1, 'N': 0}).fillna(0)

    # --- 3. INGENIERÍA DE CARACTERÍSTICAS (MEJORAS CLAVE) ---

    # Indice Tecnológico (0 a 3)
    cols_tech = ['F_TIENEINTERNET', 'F_TIENECOMPUTADOR', 'F_TIENELAVADORA']
    # Verificamos que existan antes de sumar
    exist_tech = [c for c in cols_tech if c in df.columns]
    df['TECH_SCORE'] = df[exist_tech].sum(axis=1)

    # Max Educación Padres
    if 'F_EDUCACIONPADRE' in df.columns and 'F_EDUCACIONMADRE' in df.columns:
        df['MAX_PADRES'] = df[['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']].max(axis=1)
        # Interacción: Educación x Estrato (Poderoso predictor)
        if 'F_ESTRATOVIVIENDA' in df.columns:
            df['EDU_x_ESTRATO'] = df['MAX_PADRES'] * df['F_ESTRATOVIVIENDA']

    # Interacción: Tecnología x Estrato
    if 'F_ESTRATOVIVIENDA' in df.columns:
        df['TECH_x_ESTRATO'] = df['TECH_SCORE'] * df['F_ESTRATOVIVIENDA']

    # 4. Categorías para LightGBM
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if col != 'RENDIMIENTO_GLOBAL':
            df[col] = df[col].astype('category')

    if 'ID' in df.columns:
        df = df.drop('ID', axis=1)

    return df

#Carga y procesamiento

In [7]:
print("Cargando...")
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
test_ids = test['ID']

print("Procesando...")
train_proc = preprocess_turbo(train)
test_proc = preprocess_turbo(test)

X = train_proc.drop('RENDIMIENTO_GLOBAL', axis=1)
y = train_proc['RENDIMIENTO_GLOBAL'].map(target_map)
X_test = test_proc[X.columns]

print(f"Listos. Columnas generadas: {len(X.columns)}")

Cargando...
Procesando...
Listos. Columnas generadas: 23


#Entrenamiento del modelo

In [8]:
# 4. ENTRENAMIENTO VELOZ
clf = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=4,
    n_estimators=1200,    # Reducido de 3000 a 1200 (Suficiente)
    learning_rate=0.05,   # SUBIDO: Más rápido
    num_leaves=40,        # Un poco más complejo para capturar detalles
    max_depth=-1,
    min_child_samples=30,
    subsample=0.8,
    colsample_bytree=0.7,
    random_state=42,
    n_jobs=-1
)

# Volvemos a 5 Folds (Más que suficiente y mucho más rápido)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

preds_test_folds = []
scores = []

print("Entrenando Modelo Turbo (5 Folds)...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    clf.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)] # Silencioso
    )

    acc = accuracy_score(y_val, clf.predict(X_val))
    scores.append(acc)
    preds_test_folds.append(clf.predict(X_test))

    print(f"Fold {fold+1}: {acc:.5f}")

print(f"--> Promedio: {np.mean(scores):.5f}")

Entrenando Modelo Turbo (5 Folds)...
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.233059 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1673
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 22
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[430]	valid_0's multi_logloss: 1.18838
Fold 1: 0.44139
[LightGBM] [Warning] Categorical features with more bins than the configure

#Generar Submission

In [10]:
# 5. GUARDAR
print("Generando archivo...")
preds_matrix = np.column_stack(preds_test_folds)
final_preds_idx, _ = mode(preds_matrix, axis=1)
final_preds_idx = final_preds_idx.ravel()

final_preds_str = [inv_target_map[x] for x in final_preds_idx]

submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': final_preds_str
})

submission.to_csv('submission_turbo_lgbm.csv', index=False)
print("Archivo 'submission_turbo_lgbm.csv' creado.")

Generando archivo...
Archivo 'submission_turbo_lgbm.csv' creado.


In [11]:
!kaggle competitions submit \
  -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
  -f submission_turbo_lgbm.csv \
  -m "Submission LightGBM Turbo + Interacciones"

100% 4.05M/4.05M [00:01<00:00, 3.36MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia